Futures(동시성) <br>

비동기 작업 실행의 A-> B
    동기작업이면 A를 완전히 끝낼 때까지 기다렸다가 B를 실행한다.
    비동기면 A를 하면서 B도 수행할 수 있다.

지연시간(Block): CPU 및 리소스 낭비를 방지한다. -> fileIO, networkIO에서 권장한다. <br>
비동기 작업과 적함한 프로그램일 경우 압도적으로 성능이 향상된다. <br>

2가지 패턴 <br>
futures: 비동기 실행을 위한 API를 high-level로 쉽게 사용할 수 있도록 개선된 패키지이다.<br>
1. multi-threading/multi-processing API가 통일되었다. 매우 사용하기 쉬움.
2. 실행 중인 작업 취소(interrupt), 완료 여부 체크, 타임아웃 옵션(timeout), 콜백 추가(callback), 동기화 코드(synchronized code)를 쉽게 작성할 수 있다.
-> promise 개념
- concurrent.futures map 사용법1
- concurrent.futures wait, as_completed 사용법2

GIL(Global Interpreter Lock) : 2개 이상의 thread가 동시에 실행하면서 하나의  자원을 접근하는 경우일 때, <br>
문제를 방지하기 위해 GIL이 실행된다. 리소스 전체에 락이 걸린다. <br>
Context-switching cost 증가 <br>
GIL 우회: 멀티프로세싱 사용, Cpython 사용 <br>

In [3]:
import os, time
from concurrent import futures

WORK_LIST = [10000, 100000, 1000000, 10000000]



In [4]:
# 동시성 합계 계산 메인 함수
# 누적 합계 함수(generator)

def sum_generator(n):
    return sum(n for n in range(1, n+1))

In [5]:
print(n for n in range(1, 10))
# tuple comprehension unpacking


<generator object <genexpr> at 0x0000021924B457B0>


In [6]:
def main():
    # Worker count
    worker = min(10, len(WORK_LIST))

    start_tm = time.time() # 시작

    # 결과 건수
    # ThreadPoolExecuter() 또는 ProcessPoolExecutor() 같은 방식으로 사용함
    with futures.ThreadPoolExecutor() as executor:
        result = executor.map(sum_generator, WORK_LIST)
        # 그냥 map처럼 (함수, target_list) -> list
        # map은 작업 순서를 유지하고 즉시 실행한다.
        # 모든 결과가 생성되어서야 list에 담아낸다.


    end_tm = time.time() - start_tm # 끝

    msg = '\n Result - > {} Time : {:.2f}s'

    print(msg.format(list(result), end_tm))

In [7]:
# 실행
if __name__ == '__main__':
    main()


 Result - > [50005000, 5000050000, 500000500000, 50000005000000] Time : 0.93s


In [8]:
def main2():
    # Worker count
    worker = min(10, len(WORK_LIST))

    start_tm = time.time() # 시작

    # 결과 건수
    # ThreadPoolExecuter() 또는 ProcessPoolExecutor() 같은 방식으로 사용함
    with futures.ProcessPoolExecutor(max_workers= worker) as executor:
        result = executor.map(sum_generator, WORK_LIST)
        # 그냥 map처럼 (함수, target_list)
        # map은 작업 순서를 유지하고 즉시 실행한다.



    end_tm = time.time() - start_tm # 끝

    msg = '\n Result - > {} Time : {:.2f}s'

    print(msg.format(list(result), end_tm))

multi-processing에선 CPU 사양이나 OS 환경에 따라서 max_worker의 한계가 있다.  <br>
BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.<br>
이런 오류가 발생할 수 있으니 최대 코어 수를 줄여라. 내 컴퓨터는 사양이 낮아서 안 된다. <br>
leve4의 multiprocesss를 참고하자.

In [9]:
if __name__ == '__main__':
    main2()

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

[10000, 100000, 1000000, 10000000] <br>
각각 필요한 시간이 다르고 잘 끝날 지 알 수 없다. <br>
(fs, timeout=None, return_when=ALL_COMPLETED) -> done, not_done : set <br>

FIRST_COMPLETED: 어느 하나라도 끝나거나 취소될 때 함수가 반환된다. <br>
FIRST_EXCEPTION: 어느 하나라도 예외로 완료되면 반환되며 예외 없으면 ALL_COMPLETED와 동일하다<br>
ALL_COMPLETED: 모든 future가 끝나거나 취소되면 함수가 반환된다. <br>

In [ ]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, wait, as_completed
# 따로 가져와야 한다.


def main2():
    # Worker count
    worker = min(10, len(WORK_LIST))

    start_tm = time.time() # 시작

    futures_list = []
    with futures.ThreadPoolExecutor(max_workers= worker) as executor:
        for work in WORK_LIST:
            # future만 반환하지 실행되지 않는다.
            # 미래의 할 일을 반환할 뿐이다.
            future = executor.submit(sum_generator, work)
            # scheduling
            futures_list.append(future)
            print(f'Scheduled for {work} : {future}')
            
    

        result = wait(futures_list, timeout=7)
        # 스케쥴링된 리스트를 7초 안에 끝내라, 실행 단계
        #(fs, timeout=None, return_when=ALL_COMPLETED)

        print(f'Completed Tasks : ' + str(result.done))
        # 성공한 것만 가져오게 한다.

        print(f'Pending ones after waiting {str(result.not_done)}')
        # 실패한 것들

        print([future.result() for future in result.done])
        # 결과들
 

    msg = '\n Result - > {} Time : {:.2f}s'

    end_tm = time.time() - start_tm # 끝

    print(msg.format(list(result), end_tm))



In [ ]:
main2()
print('='*100)
#type(futures_list[0]) -> <class 'concurrent.futures._base.Future'>
#type(result[0]) -> <class 'set'>

Scheduled for 10000 : <Future at 0x128f980a640 state=finished returned int>
Scheduled for 100000 : <Future at 0x128f8a13be0 state=pending>
Scheduled for 1000000 : <Future at 0x128f980ac70 state=running>
Scheduled for 10000000 : <Future at 0x128f980af70 state=pending>
Completed Tasks : {<Future at 0x128f980af70 state=finished returned int>, <Future at 0x128f980a640 state=finished returned int>, <Future at 0x128f8a13be0 state=finished returned int>, <Future at 0x128f980ac70 state=finished returned int>}
Pending ones after waiting set()
[50000005000000, 50005000, 5000050000, 500000500000]

 Result - > [{<Future at 0x128f980af70 state=finished returned int>, <Future at 0x128f980a640 state=finished returned int>, <Future at 0x128f8a13be0 state=finished returned int>, <Future at 0x128f980ac70 state=finished returned int>}, set()] Time : 1.03s


as_completed(fs, timeout=None) <br>
먼저 완료되거나 취소되는 순서대로 반환한다. generator <br>
작업량이 매우 불균일하게 분할되거나 한 작업이 오래 걸릴 경우에 먼저 되는 것들을 받아서
먼저 작업하기에 적합하다. <br>


In [ ]:
def main3():
    # Worker count
    worker = min(10, len(WORK_LIST))

    start_tm = time.time() # 시작

    futures_list = []
    
    with ThreadPoolExecutor() as executor:

        for work in WORK_LIST:

            future = executor.submit(sum_generator, work)
            # scheduling
            futures_list.append(future)

            print(f'Scheduled for {work} : {future}')

        for future in as_completed(futures_list):
            # as_completed 말그대로 먼저 끝나는 대로 반환된다.
            result = future.result()
            done = future.done()
            cancelled = future.cancelled()

            print(f'Future Result : {result}, Done : {done}')
            print(f'Future Cancelled : {cancelled}')

 

    msg = '\n Time : {:.2f}s'

    end_tm = time.time() - start_tm # 끝

    print(msg.format(end_tm))

In [ ]:
main3()

Scheduled for 10000 : <Future at 0x128f980a700 state=finished returned int>
Scheduled for 100000 : <Future at 0x128f980ab50 state=pending>
Scheduled for 1000000 : <Future at 0x128f8a13490 state=running>
Scheduled for 10000000 : <Future at 0x128f8a13af0 state=running>
Future Result : 5000050000, Done : True
Future Cancelled : False
Future Result : 50005000, Done : True
Future Cancelled : False
Future Result : 500000500000, Done : True
Future Cancelled : False
Future Result : 50000005000000, Done : True
Future Cancelled : False

 Time : 1.11s
